In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# define the resnet-18 here
class ResNet18(nn.Module):
    def __init__(self, ResidualBlock, num_class=10):
        super(ResNet18, self).__init__()
        # TODO
    
    def forward(self, x):
        # TODO
        

In [ ]:
import torch

# define the accuracy evaluation method
def evaluate_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    model.eval()
    with torch.no_grad():
        for i, (x, y) in enumerate(loader):
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        accuracy = float(num_correct / num_samples)
        print('Number of correct sample %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * accuracy))
        return accuracy

In [ ]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import copy

NUM_TRAIN = 49000

# 数据预处理，减去cifar-10数据均值
transform_normal = T.Compose([
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))
])


# 加载训练集
cifar10_train = dset.CIFAR10('数据集/data/CIFAR10', train=True, download=True, transform=transform_normal)
loader_train = DataLoader(cifar10_train, batch_size=64, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

# 加载验证集
cifar10_val = dset.CIFAR10('数据集/data/CIFAR10', train=True, download=True, transform=transform_normal)
loader_val = DataLoader(cifar10_val, batch_size=64, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

# 加载测试集
cifar10_test = dset.CIFAR10('数据集/data/CIFAR10', train=False, download=True, transform=transform_normal)
loader_test = DataLoader(cifar10_test, batch_size=64)

learning_rate = 1e-2
model = ResNet18()
optimizer_resnet = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

for e in range(10):
    for t, (x, y) in enumerate(loader_train):
        model.train()
        scores = model(x)
        loss = F.cross_entropy(scores, y)

        optimizer_resnet.zero_grad()
        loss.backward()
        optimizer_resnet.step()

        if t%100==0:
            print('Epoch %d, iter %d, loss=%.4f' % (e, t,  loss.item()))
    evaluate_accuracy(loader_val, model)

    